In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
from deeplkt.datasets.dataset import *
from torch.utils.data import DataLoader
from deeplkt.models.pure_lkt import PureLKTNet
from deeplkt.models.lkt_alexsobel import LKTAlexSobelNet

from deeplkt.models.base_model import BaseModel

# from model_learnable_lkt import LearnableLKTNet
# from evaluate_pytorch import evaluate_video_index, plot_different_results
# from torch import optim
# from model_utils import save_model, update_model, save_results, load_model
# import matplotlib.pyplot as plt
# import time
from deeplkt.utils.util import dotdict
from deeplkt.tracker.lkt_tracker import LKTTracker
from deeplkt.config import *

/home/sudeep/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sudeep/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sudeep/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sudeep/anaconda3/envs/pytorch/lib/pyt

In [3]:
#!/usr/bin/env python
# coding: utf-8


# from pytorch_practise import *

use_cuda = torch.cuda.is_available()
device = torch.device("cuda")

vot_root_dir = '../../data/VOT/'
alov_root_dir = '../../data/ALOV/'


vot = VotDataset(os.path.join(vot_root_dir,
                       'VOT_images/'),
                 os.path.join(vot_root_dir,
                       'VOT_ann/'),
                 os.path.join(vot_root_dir,
                       'VOT_results/'), 
                 device)

alov = AlovDataset(os.path.join(alov_root_dir,
                       'ALOV_images/'),
                   os.path.join(alov_root_dir,
                       'ALOV_ann/'),
                   os.path.join(alov_root_dir,
                       'ALOV_results/'), 
                       device)


# train_loader = DataLoader(alov, batch_size=1, shuffle=False)



Parsing VOT dataset...
VOT dataset parsing done.
Total number of annotations in VOT dataset = 10188
Total number of videos in VOT dataset = 25
Parsing ALOV dataset...
ALOV dataset parsing done.
Total number of videos in ALOV dataset = 307
Total number of annotations in ALOV dataset = 15877


In [4]:
BATCH_SIZE

128

In [5]:
from deeplkt.config import *
params = dotdict({
    'mode' : MODE,
    'max_iterations' : MAX_LK_ITERATIONS,
    'epsilon' : EPSILON,
    'info': "AlexSobel LKT"
})
# lr = 0.0005
# momentum = 0.5


nn = LKTAlexSobelNet(device, params)
tracker = LKTTracker(nn)
train_params = dotdict({
    'batch_size' : BATCH_SIZE,
    'val_split' : VALIDATION_SPLIT,
    'train_examples':TRAIN_EXAMPLES,
    'shuffle_train': SHUFFLE_TRAIN,
    'random_seed': RANDOM_SEED  
})

model = BaseModel(tracker, 'checkpoint', 'logs', train_params)
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)
# count_parameters(nn)



0 Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
1 ReLU(inplace)
2 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
3 Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
4 ReLU(inplace)
5 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
6 Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
7 ReLU(inplace)
8 Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
9 ReLU(inplace)
10 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
11 ReLU(inplace)
12 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)




111759222

In [10]:
model.train_model(alov)


Train dataset size =  300
Valid dataset size =  100
EPOCH =  0
Training for epoch:0
Total training batches =  3
tensor(7.7509, device='cuda:0', grad_fn=<SmoothL1LossBackward>)
0
tensor(6.9028, device='cuda:0', grad_fn=<SmoothL1LossBackward>)
1
tensor(11.5845, device='cuda:0', grad_fn=<SmoothL1LossBackward>)
2
Training time for 0 epoch = 18.51132607460022
Validation for epoch:0
Total validation batches =  1
Total validation batches =  1
Validation time for 0 epoch = 4.8152549266815186


EPOCH =  1
Training for epoch:1
Total training batches =  3
tensor(7.7659, device='cuda:0', grad_fn=<SmoothL1LossBackward>)
0
tensor(6.8488, device='cuda:0', grad_fn=<SmoothL1LossBackward>)
1


KeyboardInterrupt: 

In [2]:
model.eval_model(vot, 0)

NameError: name 'model' is not defined